In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
21,application_1579201521534_0022,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-21>

In [2]:
sc.install_pypi_package("h2o-pysparkling-2.4")
sc.install_pypi_package("h2o")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processing /var/lib/livy/.cache/pip/wheels/0d/2f/d2/7e0c36c53b57ca597430858ab3d72e1a612a10780016ff6765/h2o_pysparkling_2.4-3.28.0.1.post1-py2.py3-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e/tabulate-0.8.6-cp36-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471/pyspark-2.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c9/dc/45cdef1b4d119eb96316b3117e6d5708a08029992b2fee2c143c7a0a5cc5/colorama-0.4.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
Processing /var/lib/livy/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e/future-0.18.2-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/53/c737818eb9a7dc32a7cd4f1396e787bd94200c3997c72c1dbe0285

## H2O 

In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pysparkling import *
from h2o.automl import H2OAutoML

import pyspark
import h2o

sc = SparkContext.getOrCreate()

ss = pyspark.sql.SparkSession.builder.getOrCreate()

hc = H2OContext.getOrCreate(ss)


Sparkling Water Context:
 * Sparkling Water Version: 3.28.0.1-1-2.4
 * H2O name: sparkling-water-alaa_local-1579237463943
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (driver,10.1.128.248,54323)
  ------------------------

  Open H2O Flow in browser: http://10.1.128.248:54323 (CMD + click in Mac OSX)

    


In [5]:
fname = "Wrist/S_2.csv"
wrist = sc.textFile(fname)
wrist = ss.read.csv(wrist, header=True, inferSchema=True)

In [6]:
wrist.show(5)

+--------------------+-----+-----+-----+-----+------+--------+-----+
|                 _c0|label|ACC_0|ACC_1|ACC_2|   BVP|     EDA| TEMP|
+--------------------+-----+-----+-----+-----+------+--------+-----+
|                 0.0|  0.0| 62.0|-21.0|107.0|-59.37|1.138257|35.41|
|0.001428571428571...|  0.0| null| null| null|  null|    null| null|
|0.002857142857142857|  0.0| null| null| null|  null|    null| null|
|0.004285714285714286|  0.0| null| null| null|  null|    null| null|
|0.005714285714285714|  0.0| null| null| null|  null|    null| null|
+--------------------+-----+-----+-----+-----+------+--------+-----+
only showing top 5 rows



In [7]:
def clean_df(df, sub_id):
    df = df.filter(df.TEMP.isNotNull())
    df = df.withColumn('sub_id', lit(f'S_{sub_id}'))
    df = df.filter(df.label.isin([1,2,3]))
    return df

In [8]:
wrist = clean_df(wrist, 2)

In [9]:
wrist.show(5)

+------+-----+-----+-----+-----+------+--------+-----+------+
|   _c0|label|ACC_0|ACC_1|ACC_2|   BVP|     EDA| TEMP|sub_id|
+------+-----+-----+-----+-----+------+--------+-----+------+
|306.75|  1.0| 65.0| 12.0| 21.0|-33.58|1.640539|35.81|   S_2|
| 307.0|  1.0| 55.0|-32.0| 44.0| 30.52|1.634132|35.81|   S_2|
|307.25|  1.0| 49.0|-21.0| 30.0| 52.54|1.614912|35.81|   S_2|
| 307.5|  1.0| 52.0|-22.0| 27.0| 94.31|1.591848|35.81|   S_2|
|307.75|  1.0| 58.0|-14.0| 28.0|-38.86|1.558534|35.81|   S_2|
+------+-----+-----+-----+-----+------+--------+-----+------+
only showing top 5 rows



In [10]:
nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

In [12]:
wrists = wrist
for i in nums[1:]:
    wrist = ss.read.csv(f'Wrist/S_{i}.csv', header=True, inferSchema=True)
    wrist = clean_df(wrist, i)
    wrists = wrists.union(wrist)
wrists.cache()

DataFrame[_c0: double, label: double, ACC_0: double, ACC_1: double, ACC_2: double, BVP: double, EDA: double, TEMP: double, sub_id: string]

In [13]:
wrists.count()

132608

## Splitting

In [14]:
chest_path = 'Chest/'
wrist_path = 'Wrist/'
nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

In [15]:
chest = ss.read.csv(chest_path+'S_2.csv', header=True, inferSchema=True)

In [16]:
wrist = ss.read.csv(wrist_path+'S_2.csv', header=True, inferSchema=True)

In [17]:
def clean_df(df):
    df = df.filter(df.TEMP.isNotNull())
    return df

In [18]:
def combine(wrist, chest):
    chest = chest.drop('_c0')
    wrist = wrist.withColumnRenamed('_c0', 'col_1')
    wrist = clean_df(wrist)
    joined = chest.join(wrist, 'col_1')
    joined = joined.filter(joined.label.isin([1,2,3]))
    return joined

## Chest+Wrist

In [19]:
chest = ss.read.csv(f"{chest_path}S_2.csv", header=True, inferSchema=True)
wrist = ss.read.csv(f"{wrist_path}S_2.csv", header=True, inferSchema=True)
wrist = wrist.drop('label')
all_data = combine(wrist, chest)
all_data = all_data.filter(all_data.label.isin([1, 2, 3]))

In [20]:
for sub_id in nums[1:]:
    chest = ss.read.csv(f"{chest_path}S_{sub_id}.csv", header=True, inferSchema=True)
    wrist = ss.read.csv(f"{wrist_path}S_{sub_id}.csv", header=True, inferSchema=True)
    wrist = wrist.drop('label')
    joined = combine(wrist, chest)
    joined = joined.filter(joined.label.isin([1, 2, 3]))
    all_data = all_data.union(joined)
all_data.cache()

DataFrame[col_1: double, label: int, sub_id: string, chest_ACC_1: double, chest_ACC_2: double, chest_ACC_3: double, chest_ECG: double, chest_EMG: double, chest_EDA: double, chest_Temp: double, chest_Resp: double, ACC_0: double, ACC_1: double, ACC_2: double, BVP: double, EDA: double, TEMP: double]

In [21]:
all_data.count()

132608

In [22]:
all_data.columns

['col_1',
 'label',
 'sub_id',
 'chest_ACC_1',
 'chest_ACC_2',
 'chest_ACC_3',
 'chest_ECG',
 'chest_EMG',
 'chest_EDA',
 'chest_Temp',
 'chest_Resp',
 'ACC_0',
 'ACC_1',
 'ACC_2',
 'BVP',
 'EDA',
 'TEMP']

In [23]:
# data_split = all_data.randomSplit([.8, .2], seed=2) 
train_data = all_data.filter(all_data.sub_id != "S17").cache()
test_data = all_data.filter(all_data.sub_id == "S17").cache()

In [24]:
train_data.count()

123504

In [25]:
test_data.count()

9104

In [20]:
# test_data.write.option('path', '/testdata_d').saveAsTable('testdata_d')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# train_data = all_data.join(test_data, ['sub_id', 'col_1'], 'leftanti')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
# train_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
train_h2o = hc.as_h2o_frame(train_data, "train_data")

In [27]:
test_h2o = hc.as_h2o_frame(test_data, "test_data")

In [28]:
train_h2o.show(5)

,col_1,label,sub_id,chest_ACC_1,chest_ACC_2,chest_ACC_3,chest_ECG,chest_EMG,chest_EDA,chest_Temp,chest_Resp,ACC_0,ACC_1,ACC_2,BVP,EDA,TEMP
0,315.75,1,S2,0.8648,-0.1346,-0.3182,-0.115402,0.027420,5.186844,29.211609,-0.660706,43,-22,41,-75.90,1.477810,35.81
1,367.25,1,S2,0.6258,-0.1080,-0.7046,0.015610,0.007736,4.504395,28.978760,-1.153564,21,12,59,-17.96,1.120318,35.73
2,451.50,1,S2,0.6118,-0.1064,-0.7148,0.064133,-0.011765,3.512955,28.819489,2.272034,40,37,33,-2.50,0.561559,35.79
3,496.00,1,S2,0.6484,-0.0554,-0.6788,0.004807,-0.013275,3.224945,28.808044,-1.458740,56,23,22,-6.04,0.500055,35.87
4,548.25,1,S2,0.6510,-0.0616,-0.6818,-0.200821,-0.005539,2.758408,28.558930,-1.301575,54,26,20,13.59,0.470515,35.79
5,558.00,1,S2,0.6398,-0.0626,-0.7010,0.047287,-0.007141,2.700424,28.533203,1.930237,53,26,20,-20.01,0.447451,35.79
6,558.50,1,S2,0.6410,-0.0654,-0.7046,0.080795,0.000687,2.692413,28.556091,0.788879,55,26,20,8.54,0.446170,35.79
7,596.00,1,S2,0.6254,-0.0702,-0.6902,-0.192673,-0.016068,2.458572,28.706299,-0.204468,59,18,16,83.41,0.378260,35.87
8,608.25,1,S2,0.6290,-0.0696,-0.7110,0.051865,-0.004944,2.391815,28.647644,1.274109,11,10,62,-42.60,0.379541,35.89
9,672.25,1,S2,0.6328,-0.0654,-0.7138,0.016434,0.000824,2.125168,28.397430,-0.512695,57,-6,-1,108.84,0.291129,35.79


In [30]:
train_h2o.types

{'col_1': 'real',
 'label': 'int',
 'sub_id': 'string',
 'chest_ACC_1': 'real',
 'chest_ACC_2': 'real',
 'chest_ACC_3': 'real',
 'chest_ECG': 'real',
 'chest_EMG': 'real',
 'chest_EDA': 'real',
 'chest_Temp': 'real',
 'chest_Resp': 'real',
 'ACC_0': 'int',
 'ACC_1': 'int',
 'ACC_2': 'int',
 'BVP': 'real',
 'EDA': 'real',
 'TEMP': 'real'}

In [37]:
train_h2o["label"] = train_h2o["label"].asfactor()
train_h2o["sub_id"] = train_h2o["sub_id"].asfactor()

In [36]:
test_h2o["label"] = test_h2o["label"].asfactor()
test_h2o["sub_id"] = test_h2o["sub_id"].asfactor()

In [38]:
predictors = train_h2o.names[:]
predictors.remove("col_1")
response = "label"
predictors.remove(response)

In [39]:
model_automl = H2OAutoML(max_runtime_secs = 1000, seed = 2, max_models = 10)

In [ ]:
model_automl.train(x=predictors,
                   y=response,
                   training_frame=train_h2o,
                   validation_frame=test_h2o,
                   fold_column="sub_id")
print(model_automl.leaderboard)

AutoML progress: |
21:17:16.273: Fold column sub_id will be used for cross validation. nfolds parameter will be ignored.

████████████████████████████████

In [74]:
h2o.get_model("XGBoost_1_AutoML_20200117_010755")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20200117_010755


Model Summary: 
    number_of_trees
--  -----------------
    91


ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.00019910704882845388
RMSE: 0.014110529714665353
LogLoss: 0.011401406864344762
Mean Per-Class Error: 5.7065034183882365e-05

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
1      2      3      Error        Rate
-----  -----  -----  -----------  -----------
65720  0      0      0            0 / 65,720
1      36971  0      2.70475e-05  1 / 36,972
3      0      20809  0.000144148  3 / 20,812
65724  36971  20809  3.23876e-05  4 / 123,504

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.999968
2    1
3    1

ModelMetricsMultinomial: xgboost
** Reported on validation data. **

MSE: 0.3408622426422221
RMSE: 0.5838340882838395
LogLoss: 1.1751345798810509
Mean Per-Class Error: 0.617278669607029

Confusion Matrix: Row labels: Actual 

In [70]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

rf_model = H2ORandomForestEstimator(ntrees=200)
rf_model.train(x=predictors,
               y=response,
               training_frame=train_h2o,
               validation_frame=test_h2o)
rf_model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

drf Model Build progress: [###############################################] 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1579219406050_2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    200                600                         2.6102e+06             13           20           18.4217       147           767           341.935


ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.00022886419591212965
RMSE: 0.015128258191613788
LogLoss: 0.00300436814261544
Mean Per-Class Error: 2.1088423517075277e-05

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
1      2      3      Error        Rate
-----  -----  -----  -

In [71]:
rf_model.model_performance

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1579219406050_2


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    200                600                         2.6102e+06             13           20           18.4217       147           767           341.935


ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.00022886419591212965
RMSE: 0.015128258191613788
LogLoss: 0.00300436814261544
Mean Per-Class Error: 2.1088423517075277e-05

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
1      2      3      Error        Rate
-----  -----  -----  -----------  -----------
65719  1      0      1.52161e-05  1 / 65,720
0      36972

## Wrist-only

In [85]:
wrists.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(_c0=306.75, label=1.0, ACC_0=65.0, ACC_1=12.0, ACC_2=21.0, BVP=-33.58, EDA=1.640539, TEMP=35.81, sub_id='S_2')

In [86]:
wrists_data = wrists

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
wrists_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

132608

In [90]:
wrists_train_data = wrists_data.filter(wrists_data.sub_id != "S_17").cache()
wrists_test_data = wrists_data.filter(wrists_data.sub_id == "S_17").cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
wrists_train_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

123504

In [92]:
wrists_test_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9104

In [94]:
wrist_train_h2o = hc.as_h2o_frame(wrists_train_data, "wrists_train_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
wrist_test_h2o = hc.as_h2o_frame(wrists_test_data, "wrists_test_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
wrist_train_h2o["label"] = wrist_train_h2o["label"].asfactor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
wrist_test_h2o["label"] = wrist_test_h2o["label"].asfactor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
wrists_predictors = wrist_train_h2o.names[:]
wrists_predictors.remove("_c0")
wrists_response = "label"
wrists_predictors.remove(wrists_response)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
wrist_train_h2o.types

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'_c0': 'real', 'label': 'enum', 'ACC_0': 'int', 'ACC_1': 'int', 'ACC_2': 'int', 'BVP': 'real', 'EDA': 'real', 'TEMP': 'real', 'sub_id': 'string'}

In [101]:
wrists_model_automl = H2OAutoML(nfolds = 3, max_runtime_secs = 1000, seed = 2, max_models = 10)
wrists_model_automl.train(x=wrists_predictors,
                   y=wrists_response,
                   training_frame=wrist_train_h2o,
                   validation_frame=wrist_test_h2o)
print(wrists_model_automl.leaderboard)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AutoML progress: [########################################################] 100%
model_id                                               mean_per_class_error     logloss       rmse          mse
---------------------------------------------------  ----------------------  ----------  ---------  -----------
XGBoost_1_AutoML_20200117_015544                                0.000558105  0.00284161  0.0192358  0.000370017
XGBoost_2_AutoML_20200117_015544                                0.0015964    0.0304554   0.0498706  0.00248707
XGBoost_3_AutoML_20200117_015544                                0.0859506    0.885997    0.587115   0.344704
StackedEnsemble_BestOfFamily_AutoML_20200117_015544             0.666667     0.996842    0.616396   0.379944
StackedEnsemble_AllModels_AutoML_20200117_015544                0.666667     0.996842    0.616396   0.379944

[5 rows x 5 columns]

In [103]:
h2o.get_model("XGBoost_1_AutoML_20200117_015544")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20200117_015544


Model Summary: 
    number_of_trees
--  -----------------
    281


ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 8.144032583696577e-05
RMSE: 0.009024429391211711
LogLoss: 0.001377534804467871
Mean Per-Class Error: 5.7065034183882365e-05

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
1      2      3      Error        Rate
-----  -----  -----  -----------  -----------
65720  0      0      0            0 / 65,720
1      36971  0      2.70475e-05  1 / 36,972
3      0      20809  0.000144148  3 / 20,812
65724  36971  20809  3.23876e-05  4 / 123,504

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.999968
2    1
3    1

ModelMetricsMultinomial: xgboost
** Reported on validation data. **

MSE: 0.4205573746592789
RMSE: 0.6485039511516325
LogLoss: 1.9337256490223413
Mean Per-Class Error: 0.5970505026372303

Confusion Matrix: Row labels: Actual

## Chest-only

In [137]:
chests_data = all_data.drop('ACC_1', 'ACC_2', 'ACC_0', 'BVP', 'EDA', 'TEMP')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
chests_data = chests_data.filter(chests_data.label.isin([1,2,3]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
chests_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

132608

In [140]:
chests_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----+------+------------------+--------------------+-------------------+--------------------+-------------------+------------------+----------+-----------------+
| col_1|label|sub_id|       chest_ACC_1|         chest_ACC_2|        chest_ACC_3|           chest_ECG|          chest_EMG|         chest_EDA|chest_Temp|       chest_Resp|
+------+-----+------+------------------+--------------------+-------------------+--------------------+-------------------+------------------+----------+-----------------+
|315.75|    1|    S2| 0.864799976348877| -0.1345999836921692|-0.3181999921798706| -0.1154022216796875| 0.0274200439453125|5.1868438720703125| 29.211609|-0.66070556640625|
|367.25|    1|    S2|0.6258000135421753|-0.10799998044967651|-0.7045999765396118|0.015609741210937498| 0.0077362060546875|     4.50439453125|  28.97876|  -1.153564453125|
| 451.5|    1|    S2| 0.611799955368042| -0.1064000129699707|-0.7148000001907349|  0.0641326904296875|-0.0117645263671875|3.5129547119140625| 28.

In [141]:
chests_train_data = chests_data.filter(chests_data.sub_id != "S17").cache()
chests_test_data = chests_data.filter(chests_data.sub_id == "S17").cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
chest_train_h2o = hc.as_h2o_frame(chests_train_data, "chests_train_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
chest_test_h2o = hc.as_h2o_frame(chests_test_data, "chests_test_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
chest_train_h2o["label"] = chest_train_h2o["label"].asfactor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
chest_test_h2o["label"] = chest_test_h2o["label"].asfactor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
chest_train_h2o.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  col_1    label  sub_id      chest_ACC_1    chest_ACC_2    chest_ACC_3    chest_ECG     chest_EMG    chest_EDA    chest_Temp    chest_Resp
-------  -------  --------  -------------  -------------  -------------  -----------  ------------  -----------  ------------  ------------
 315.75        1  S2               0.8648        -0.1346        -0.3182  -0.115402     0.02742          5.18684       29.2116     -0.660706
 367.25        1  S2               0.6258        -0.108         -0.7046   0.0156097    0.00773621       4.50439       28.9788     -1.15356
 451.5         1  S2               0.6118        -0.1064        -0.7148   0.0641327   -0.0117645        3.51295       28.8195      2.27203
 496           1  S2               0.6484        -0.0554        -0.6788   0.00480652  -0.0132751        3.22495       28.808      -1.45874
 548.25        1  S2               0.651         -0.0616        -0.6818  -0.200821    -0.00553894       2.75841       28.5589     -1.30157
 558           1  S2    

In [147]:
chest_test_h2o.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  col_1    label  sub_id      chest_ACC_1    chest_ACC_2    chest_ACC_3    chest_ECG     chest_EMG    chest_EDA    chest_Temp    chest_Resp
-------  -------  --------  -------------  -------------  -------------  -----------  ------------  -----------  ------------  ------------
 151.25        1  S17              0.3862        -0.0696        -0.891   -0.088028    -0.00920105       5.33638       32.9088     -2.97241
 168.5         1  S17              0.3748        -0.0802        -0.8902  -0.136642     0.00883484       5.31349       32.9538      2.37885
 224.75        1  S17              0.3702        -0.0552        -0.8952   0.0102997    0.0110321        5.32417       33.0139     -2.1759
 299           1  S17              0.3722        -0.0582        -0.8908   0.00892639  -0.00485229       5.29366       33.1341      2.19879
 305           1  S17              0.3722        -0.0602        -0.8936   0.00196838  -0.0135956        5.29289       33.113      -0.286865
 315.75        1  S17    

In [148]:
chests_predictors = chest_train_h2o.names[:]
chests_predictors.remove("col_1")
chests_response = "label"
chests_predictors.remove(chests_response)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [149]:
chests_model_automl = H2OAutoML(nfolds = 3, max_runtime_secs = 1000, seed = 2, max_models = 10)
chests_model_automl.train(x=chests_predictors,
                   y=chests_response,
                   training_frame=chest_train_h2o,
                   validation_frame=chest_test_h2o)
print(chests_model_automl.leaderboard)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

AutoML progress: [########################################################] 100%
model_id                            mean_per_class_error    logloss       rmse         mse
--------------------------------  ----------------------  ---------  ---------  ----------
XGBoost_1_AutoML_20200117_024657              0.00422862  0.0627999  0.0854367  0.00729943

[1 row x 5 columns]

In [186]:
chest_model = h2o.get_model("XGBoost_1_AutoML_20200117_024657")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
perf = chest_model.model_performance(chest_test_h2o)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [209]:
mpce = chest_model.mean_per_class_error(valid=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [233]:
chest_cols = chests_data.columns
chest_cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['col_1', 'label', 'sub_id', 'chest_ACC_1', 'chest_ACC_2', 'chest_ACC_3', 'chest_ECG', 'chest_EMG', 'chest_EDA', 'chest_Temp', 'chest_Resp']

In [234]:
chest_cols.remove("col_1")
chest_cols.remove("label")
chest_cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['sub_id', 'chest_ACC_1', 'chest_ACC_2', 'chest_ACC_3', 'chest_ECG', 'chest_EMG', 'chest_EDA', 'chest_Temp', 'chest_Resp']

In [245]:
baseline_m = mpce
imps = list()
for col in chest_cols:
    X_train = chest_train_h2o.drop(col)
    X_test = chest_test_h2o.drop(col)
    #create autoMl object
    chests_predictors = X_train.names[:]
    chests_response = "label"
    chests_model_automl = H2OAutoML(nfolds = 3, max_runtime_secs = 60, seed = 2, max_models = 10)
    chests_model_automl.train(x=chests_predictors,
                       y=chests_response,
                       training_frame=X_train,
                       validation_frame=X_test)
    leader_model = chests_model_automl.leader 
    
    mpce_new = leader_model.mean_per_class_error(valid=True)
    imp.append(baseline_m - mpce_new)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'NoneType' object has no attribute 'mean_per_class_error'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'mean_per_class_error'



In [206]:
chest_model.mean_per_class_error(valid=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.45662438431770286

In [207]:
perf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 0.23447813506858667
RMSE: 0.48422942400125446
LogLoss: 0.8584096627664244
Mean Per-Class Error: 0.45662438431770286

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
1     2     3    Error       Rate
----  ----  ---  ----------  -------------
4718  6     0    0.00127011  6 / 4,724
1066  1826  0    0.368603    1,066 / 2,892
1194  294   0    1           1,488 / 1,488
6978  2126  0    0.281195    2,560 / 9,104

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.718805
2    0.836226
3    1

In [ ]:
automl = H2OAutoML(nfolds = 3, max_runtime_secs = 200, seed = 2, max_models = 10)
automl.train(x=chests_predictors,
                   y=chests_response,
                   training_frame=chest_train_h2o,
                   validation_frame=chest_test_h2o)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [247]:
automl.leader

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…